In [30]:
!pip install scikit-learn
!pip install kfp
from typing import NamedTuple
import pandas as pd 
import pickle
import tensorflow as tf
import numpy as np 

from kfp import dsl

from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import time
import numpy as np


  Using cached scikit_learn-1.5.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.4 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [59]:
#test 
def read_data() -> list:
    data = [1, 2, 3, 4, 5]
    return data


from typing import NamedTuple

import kfp 
import kfp
from kfp import dsl
from kfp import components


def read_file() -> None:
    
    def zscore_normalization(df, name):
        mean = df[name].mean()
        sd = df[name].std()
        df[name] = (df[name] - mean) / sd

    def encode_text(df, name):
        from sklearn.preprocessing import OrdinalEncoder
        enc = OrdinalEncoder()
        data = enc.fit_transform(df[name].values.reshape(-1,1))
        df[name] = data.flatten()

    def column_names():
        f = open("kddcup.names.txt")
        s = f.read()
        arr = s.split("\n")[1:-1]
        cols = [a[0:a.index(":")] for a in arr]
        cols.append("outcome")
        return cols

    def preprocess(df):
        #df.columns = column_names()
        df.columns = [
        'duration',
        'protocol_type',
        'service',
        'flag',
        'src_bytes',
        'dst_bytes',
        'land',
        'wrong_fragment',
        'urgent',
        'hot',
        'num_failed_logins',
        'logged_in',
        'num_compromised',
        'root_shell',
        'su_attempted',
        'num_root',
        'num_file_creations',
        'num_shells',
        'num_access_files',
        'num_outbound_cmds',
        'is_host_login',
        'is_guest_login',
        'count',
        'srv_count',
        'serror_rate',
        'srv_serror_rate',
        'rerror_rate',
        'srv_rerror_rate',
        'same_srv_rate',
        'diff_srv_rate',
        'srv_diff_host_rate',
        'dst_host_count',
        'dst_host_srv_count',
        'dst_host_same_srv_rate',
        'dst_host_diff_srv_rate',
        'dst_host_same_src_port_rate',
        'dst_host_srv_diff_host_rate',
        'dst_host_serror_rate',
        'dst_host_srv_serror_rate',
        'dst_host_rerror_rate',
        'dst_host_srv_rerror_rate',
        'outcome'
        ]
        for col in df.columns:
            t = (df[col].dtype)
            if t == int or t == float:
                zscore_normalization(df, col)
            else:
                encode_text(df, col)
        df.dropna(inplace=True,axis=1)
        for col in df.columns:
            if len(df[col].unique()) == 1:
                df.drop(col, inplace=True,axis=1)

        df.loc[df['outcome'] != "normal.", 'outcome']  = 1
        df.loc[df['outcome'] == "normal.", 'outcome']  = 0

        correlation = df.corrwith(df['outcome'])
        for num in correlation:
            if num >= -0.05 and num <= 0.05:
                df.drop(df.columns[row], axis=1, inplace=True)
        return df
    
    
    from sklearn.model_selection import train_test_split
    import os 
    import pandas as pd
    import numpy as np 
    
    file_path = 'https://raw.githubusercontent.com/tsimhadri-ews/internproject/intrusion-detection-0/src/kddcup.data_10_percent_corrected.csv'
    df = pd.read_csv(file_path)
    df = preprocess(df)
    

    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["outcome"]), df["outcome"], test_size=0.2)
    
    folder_path = './data'

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")
    
    
    
    np.save(f'data/X_train.npy', X_train)
    os.rename(f'data/X_train.npy', X_train_artifact.path)
    np.save(f'data/X_test.npy', X_test)
    os.rename("/tmp/X_test.npy", X_test_artifact.path)
    np.save(f'data/y_train.npy', y_train)
    os.rename(f'data/y_train.npy', y_train_artifact.path)
    np.save(f'data/y_test.npy', y_test)
    os.rename(f'data/y_test.npy', y_test_artifact.path)
    
    metrics.log_metric("Len x_train", X_train.shape[0])
    metrics.log_metric("Len x_test", X_test.shape[0])
    
    
    print(X_train, X_test, y_train, y_test)
    return X_train, X_test, y_train, y_test
    

#read_file()



In [52]:



def train_op() -> None:
    import pickle
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, f1_score
    import numpy as np 
    import json
    import os 
    
    # Convert JSON strings back to DataFrames
    """
    X_train = pd.read_json(X_train_json)
    X_test = pd.read_json(X_test_json)
    y_train = pd.read_json(y_train_json, typ='series')
    y_test = pd.read_json(y_test_json, typ='series')
    
    X_train = np.load(f'data/X_train.npy',allow_pickle=True)
    X_test = np.load(f'data/X_test.npy',allow_pickle=True)
    y_train = np.load(f'data/y_train.npy',allow_pickle=True)
    y_test = np.load(f'data/y_train.npy',allow_pickle=True)
    """
    X_train = np.load(X_train.path) 
    X_test = np.load(X_test.path)
    y_train = np.load(y_train.path) 
    y_test = np.load(y_test.path)
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    
    #Kubeflox metrics export
    
    metrics.log_metric("Test accuracy", accuracy)
    metrics.log_metric("F1 Score", f1)
    
    #implement TFServing 

    with open(f'model.pkl', 'wb') as f:
        pickle.dump(model, f)


In [99]:
import pickle
import pandas as pd
import numpy as np

def predict_op(input_data:str) -> str:
    with open('model.pkl', 'rb') as f:
        model = pickle.load(f)
    return str(model.predict(test_data))

In [56]:
from kfp import components


#preprocess_op = components.func_to_container_op(func=preprocess_op, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn'])

read_csv_op = components.func_to_container_op(func=read_file, output_component_file='read.yaml', base_image='python:3.7', packages_to_install=['pandas','scikit-learn', 'kfp', 'numpy'])
#preprocess_op = components.func_to_container_op(func=preprocess_op, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn'])
#preprocess_op = components.func_to_container_op(func=preprocess_op, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn'])
train_op = components.func_to_container_op(func=train_op, output_component_file='train.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn','numpy' ])
#predict_op = components.func_to_container_op(func=predict_op,output_component_file='predict.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn', 'numpy'])



In [58]:
import kfp
from kfp import dsl

#read_data_op = kfp.components.load_component_from_file('read.yaml')
preprocess_data_op = kfp.components.load_component_from_file('preprocess.yaml')
train_op = kfp.components.load_component_from_file('train.yaml')
#predict_op = kfp.components.load_component_from_file('predict.yaml')


@dsl.pipeline(
    name='Machine Learning Pipeline',
    description='A pipeline to preprocess, train, and predict using sklearn and tensorflow'
)





def ml_pipeline():  #(file_path:str)
    #read_csv = read_csv_op()
    preprocess = read_csv_op()
    train = train_op()
    #predict_task = predict_op(preprocess.outputs['X_test'])

    
    
# Compile the pipeline
kfp.compiler.Compiler().compile(ml_pipeline, 'pipeline22.yaml')

    

In [35]:
from kfp import compiler

pipeline_func = ml_pipeline
pipeline_filename = pipeline_func.__name__ + '.yaml'

compiler.Compiler().compile(pipeline_func, pipeline_filename)